<a href="https://colab.research.google.com/github/donghyundavidchoi/Data_Analysis/blob/main/%EB%AA%A8%EB%8D%B8%EB%A7%81/%ED%83%80%EC%9D%B4%ED%83%80%EB%8B%89%EC%83%9D%EC%A1%B4%EC%97%AC%EB%B6%80%EC%98%88%EC%B8%A1%EB%AA%A8%EB%8D%B8_LogisticRegression_fillna_groupby_get_dummies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 타이타닉생존여부예측모델

In [1]:
!gdown --fuzzy https://drive.google.com/file/d/1txRVXp-Eh222PzIeTQ-fuAoUYFR9f7q-/view?usp=sharing
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Downloading...
From: https://drive.google.com/uc?id=1txRVXp-Eh222PzIeTQ-fuAoUYFR9f7q-
To: /content/kaggle.json
100% 73.0/73.0 [00:00<00:00, 119kB/s]


In [2]:
!kaggle competitions download -c titanic

  0% 0.00/34.1k [00:00<?, ?B/s]
100% 34.1k/34.1k [00:00<00:00, 14.0MB/s]


In [3]:
!unzip /content/titanic.zip

Archive:  /content/titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [124]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("/content/train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Survived', id_name='PassengerId')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 11), (179, 11), (712, 2), (179, 2))

------

# EDA

In [125]:
X_train

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
90,91,3,"Christmann, Mr. Emil",male,29.0,0,0,343276,8.0500,NaN,S
103,104,3,"Johansson, Mr. Gustaf Joel",male,33.0,0,0,7540,8.6542,NaN,S
577,578,1,"Silvey, Mrs. William Baird (Alice Munger)",female,39.0,1,0,13507,55.9000,E44,S
215,216,1,"Newell, Miss. Madeleine",female,31.0,1,0,35273,113.2750,D36,C
191,192,2,"Carbines, Mr. William",male,19.0,0,0,28424,13.0000,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
621,622,1,"Kimball, Mr. Edwin Nelson Jr",male,42.0,1,0,11753,52.5542,D19,S
128,129,3,"Peter, Miss. Anna",female,NaN,1,1,2668,22.3583,F E69,C
57,58,3,"Novel, Mr. Mansouer",male,28.5,0,0,2697,7.2292,NaN,C
341,342,1,"Fortune, Miss. Alice Elizabeth",female,24.0,3,2,19950,263.0000,C23 C25 C27,S


In [126]:
X_train.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,712.000000,712.000000,575.000000,712.000000,712.000000,712.000000
mean,446.910112,2.285112,29.414783,0.533708,0.391854,33.388155
std,257.882594,0.842875,14.589601,1.099284,0.802311,50.807818
min,1.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,226.750000,1.000000,20.000000,0.000000,0.000000,7.925000
50%,444.000000,3.000000,28.000000,0.000000,0.000000,15.047900
75%,672.250000,3.000000,37.000000,1.000000,0.000000,31.387500
max,891.000000,3.000000,74.000000,8.000000,6.000000,512.329200


In [127]:
X_train.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age            137
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          542
Embarked         1
dtype: int64

In [128]:
X_train.nunique()

PassengerId    712
Pclass           3
Name           712
Sex              2
Age             84
SibSp            7
Parch            7
Ticket         565
Fare           234
Cabin          131
Embarked         3
dtype: int64

---------

# PassengerId 별도 저장
# 생존 여부와 관련 없어 보이는 요소들 모두 삭제

In [129]:
X_train_Id = X_train['PassengerId']
X_test_Id = X_test['PassengerId']
X_train = X_train.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)
X_test = X_test.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)

In [130]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare
90,3,male,29.0,0,0,8.0500
103,3,male,33.0,0,0,8.6542
577,1,female,39.0,1,0,55.9000
215,1,female,31.0,1,0,113.2750
191,2,male,19.0,0,0,13.0000
...,...,...,...,...,...,...
621,1,male,42.0,1,0,52.5542
128,3,female,NaN,1,1,22.3583
57,3,male,28.5,0,0,7.2292
341,1,female,24.0,3,2,263.0000


-------

# get_dummies를 활용하여 Sex에 대해 원핫인코딩 진행.
# 남녀로만 나뉘므로 drop_first를 통해 한 컬럼으로 표현하도록 작성

In [131]:
X_train['Sex'] = pd.get_dummies(X_train['Sex'], drop_first=True)
X_test['Sex'] = pd.get_dummies(X_test['Sex'], drop_first=True)

In [132]:
X_train.groupby(['Pclass', 'Sex']).mean()

Age     SibSp     Parch        Fare
Pclass Sex                                           
1      0    34.400000  0.567901  0.456790  110.872377
       1    40.178025  0.303030  0.323232   66.418181
2      0    29.629310  0.474576  0.610169   22.104380
       1    30.556829  0.333333  0.222222   19.313009
3      0    19.730769  1.054054  0.792793   16.842043
       1    26.447363  0.474265  0.242647   12.148925

---------

# Age의 결측치는 Pclass와 Sex에 대해 groupby 실행 후 평균 값 입력

In [133]:
X_train['Age'] = X_train.groupby(['Pclass', 'Sex'])['Age'].apply(lambda x : x.fillna(x.mean()))
X_test['Age'] = X_test.groupby(['Pclass', 'Sex'])['Age'].apply(lambda x : x.fillna(x.mean()))

In [134]:
X_train.isna().sum()

Pclass    0
Sex       0
Age       0
SibSp     0
Parch     0
Fare      0
dtype: int64

-------

# 상관관계 확인

In [135]:
X_train.corr()

,Pclass,Sex,Age,SibSp,Parch,Fare
Pclass,1.000000,0.145065,-0.406227,0.093593,0.011343,-0.563574
Sex,0.145065,1.000000,0.121386,-0.152645,-0.229699,-0.218511
Age,-0.406227,0.121386,1.000000,-0.273330,-0.212764,0.114495
SibSp,0.093593,-0.152645,-0.273330,1.000000,0.430720,0.160399
Parch,0.011343,-0.229699,-0.212764,0.430720,1.000000,0.231739
Fare,-0.563574,-0.218511,0.114495,0.160399,0.231739,1.000000


SibSp와 Parch의 성격이 비슷한 감이 있기에 상관관계에 영향이 더 큰 Parch 삭제

In [136]:
X_train = X_train.drop(columns='Parch', axis=1)
X_test = X_test.drop(columns='Parch', axis=1)

In [138]:
X_train

,Pclass,Sex,Age,SibSp,Fare
90,3,1,29.000000,0,8.0500
103,3,1,33.000000,0,8.6542
577,1,0,39.000000,1,55.9000
215,1,0,31.000000,1,113.2750
191,2,1,19.000000,0,13.0000
...,...,...,...,...,...
621,1,1,42.000000,1,52.5542
128,3,0,19.730769,1,22.3583
57,3,1,28.500000,0,7.2292
341,1,0,24.000000,3,263.0000


-------------

# y_train 확인

In [137]:
y_train

,PassengerId,Survived
90,91,0
103,104,0
577,578,1
215,216,1
191,192,0
...,...,...
621,622,1
128,129,1
57,58,0
341,342,1


---------

# 모델링
# 결과가 두 종류로 나뉘어 지는 분류 문제이므로 해당 문제 해결에 가볍고 빠른 LogisticRegression 선택

In [139]:
from sklearn.linear_model import LogisticRegression

In [140]:
model = LogisticRegression()
model.fit(X_train, y_train['Survived'])
print(model.score(X_train, y_train['Survived']))

0.8202247191011236


------------

# X_test의 예측값 csv파일로 저장

In [141]:
prediction = model.predict_proba(X_test)

In [142]:
output = pd.DataFrame({'PassengerId': X_test_Id, 'Survived' : prediction[:,1]})

In [143]:
output

,PassengerId,Survived
210,211,0.100788
876,877,0.120131
666,667,0.277520
819,820,0.072028
736,737,0.292502
...,...,...
494,495,0.114883
871,872,0.849798
530,531,0.931615
157,158,0.077386


In [144]:
output.to_csv('20220530.csv',index=False)

In [145]:
data = pd.read_csv('/content/20220530.csv')

In [146]:
data

,PassengerId,Survived
0,211,0.100788
1,877,0.120131
2,667,0.277520
3,820,0.072028
4,737,0.292502
...,...,...
174,495,0.114883
175,872,0.849798
176,531,0.931615
177,158,0.077386


-------

# 최종 점수

In [147]:
round(model.score(X_test, y_test['Survived'])*100, 2)

77.65